In [1]:
import tensorflow as tf
import gzip
import os
from  six.moves import urllib
import numpy as np


SOURCE_URL = "http://yann.lecun.com/exdb/mnist/"
DATA_DIRECTORY ="data"

image_size = 28
num_channels = 1
pixel_depth = 255
num_labels = 10
validation_size = 5000

def maybe_download(filename):
    if not tf.gfile.Exists(DATA_DIRECTORY):
        tf.gfile.MakeDirs(DATA_DIRECTORY)
    filepath = os.path.join(DATA_DIRECTORY, filename)

    if not tf.gfile.Exists(filepath):
        filepath,_ = urllib.request.urlretrieve(SOURCE_URL + filename, filepath)  #urlretrieve()方法直接將遠程數據下載到本地
        with tf.gfile.GFile(filepath) as f:
            size  = f.size()
        print('Successfully downloaded', filename, size, 'bytes.')
    return filepath

def extract_data(filename,num_images,norm_scale = True):
    print('Extracting', filename)

    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(image_size*image_size*num_images*num_channels)
        data = np.frombuffer(buf,dtype = np.uint8).astype(np.float32)
        if norm_scale:
            data = data/pixel_depth
        data = data.reshape(num_images,image_size,image_size,num_channels)
        data = np.reshape(data,[num_images,-1])
        return data

def  extract_labels(filename,num_images):

    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * num_images)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
        num_labels_data = len(labels)
        one_hot_encoding = np.zeros((num_labels_data,num_labels))
        one_hot_encoding[np.arange(num_labels_data),labels] = 1
        one_hot_encoding = np.reshape(one_hot_encoding, [-1, num_labels])
    return one_hot_encoding

def prepare_MNIST_Data():
    train_data_filename = maybe_download('train-images-idx3-ubyte.gz')
    train_labels_filename = maybe_download('train-labels-idx1-ubyte.gz')
    test_data_filename = maybe_download('t10k-images-idx3-ubyte.gz')
    test_labels_filename = maybe_download('t10k-labels-idx1-ubyte.gz')

    train_data = extract_data(train_data_filename,60000)
    train_labels = extract_labels(train_labels_filename,60000)
    test_data = extract_data(test_data_filename,10000)
    test_labels = extract_labels(test_labels_filename,10000)

    validation_data = train_data[:validation_size,:]
    validation_labels = train_labels[:validation_size,:]
    train_data = train_data[validation_size:,:]
    train_labels = train_labels[validation_size:,:]

    return train_data,train_labels,validation_data,validation_labels,test_data,test_labels

def preprocessing_data(data):
    
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    config = tf.ConfigProto(allow_soft_placement = True)
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.5)
    config.gpu_options.allow_growth = True
 
    sess = tf.InteractiveSession(config = config)
    
#     sess = tf.Session()
    sess.run(tf.initialize_all_variables())
    data = tf.reshape(data, [-1,28,28,1])
    train_data = tf.image.resize_images(data, [64,64])
    train_data = (train_data - 0.5) / 0.5
    train_data = tf.reshape(train_data,[-1,64*64*1])
    train_set = sess.run(train_data)
    sess.close()
    return train_set

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import os

class Gan():  
    #Gan(sess,[64,64,1],FLAGS.z_d, FLAGS.learning_rate, FLAGS.batch_size)
    #Gan(sess,[64,64,1],100, 0.00004, 64)

    def __init__(self,sess,data_shape,z_d, learning_rate,batch_size):
        self.sess  = sess
        self.data_shape = data_shape #[64,64,1]
        self.length = self.data_shape[0]*self.data_shape[1]*self.data_shape[2] #64*64*1 = 4096
        self.z_d = z_d #100
        self.learning_rate = learning_rate  #0.00004
        self.batch_size = batch_size #64
        self.beta1 = 0.5
        self.build_net()

    def Generator(self, z, is_training, reuse):
        depths = [1024, 512, 256, 128] + [self.data_shape[2]]
        with tf.variable_scope("Generator", reuse = reuse):
            with tf.variable_scope("g_fc1", reuse = reuse):
                output = tf.layers.dense(z, depths[0]*4*4, trainable = is_training)
                output = tf.reshape(output, [self.batch_size, 4, 4, depths[0]])
                output = tf.nn.relu(tf.layers.batch_normalization(output, training = is_training))

            with tf.variable_scope("g_dc1", reuse = reuse):
                output = tf.layers.conv2d_transpose(output, depths[1], [5,5], strides =(2,2), padding ="SAME", trainable = is_training)
                output = tf.nn.relu(tf.layers.batch_normalization(output, training = is_training))

            with tf.variable_scope("g_dc2", reuse = reuse):
                output = tf.layers.conv2d_transpose(output, depths[2], [5,5], strides = (2,2), padding ="SAME", trainable = is_training)
                output = tf.nn.relu(tf.layers.batch_normalization(output, training = is_training))

            with tf.variable_scope("g_dc3", reuse = reuse):
                output = tf.layers.conv2d_transpose(output,depths[3], [5,5], strides = (2,2), padding ="SAME", trainable = is_training)
                output = tf.nn.relu(tf.layers.batch_normalization(output, training = is_training))

            with tf.variable_scope("g_dc4", reuse = reuse):
                output = tf.layers.conv2d_transpose(output,depths[4], [5,5], strides = (2,2), padding = "SAME", trainable = is_training)
                g_logits = tf.nn.tanh(output)

        return g_logits

    def Discriminator(self,X, is_training, reuse):
        depths = [self.data_shape[2]] + [64, 128, 256, 512]   # [1,64,128,256,512]
        with tf.variable_scope("Discriminator", reuse = reuse):
            with tf.variable_scope("d_cv1", reuse = reuse):
                output = tf.layers.conv2d(X, depths[1], [5,5], strides = (2,2), padding ="SAME", trainable = is_training)
                output = tf.nn.leaky_relu(tf.layers.batch_normalization(output, training = is_training))

            with tf.variable_scope("d_cv2", reuse = reuse):
                output = tf.layers.conv2d(output, depths[2], [5,5], strides = (2,2), padding ="SAME", trainable = is_training)
                output = tf.nn.leaky_relu(tf.layers.batch_normalization(output, training = is_training))

            with tf.variable_scope("d_cv3", reuse = reuse):
                output = tf.layers.conv2d(output, depths[3], [5,5], strides = (2,2), padding = "SAME", trainable = is_training)
                output = tf.nn.leaky_relu(tf.layers.batch_normalization(output, training = is_training))

            with tf.variable_scope("d_cv4", reuse = reuse):
                output = tf.layers.conv2d(output, depths[4], [5,5], strides = (2,2), padding ="SAME", trainable = is_training)
                output = tf.nn.leaky_relu(tf.layers.batch_normalization(output, training = is_training))

            with tf.variable_scope("d_fc1", reuse = reuse):
                output = tf.layers.flatten(output)
                d_logits = tf.layers.dense(output,1, trainable = is_training)

            return d_logits

    def plot_and_save(self, order, images):
        batch_size = len(images)
        n = np.int(np.sqrt(batch_size))
        image_size = np.shape(images)[2]
        n_channel = np.shape(images)[3]
        images = np.reshape(images, [-1,image_size,image_size,n_channel])
        canvas = np.empty((n * image_size, n * image_size))
        for i in range(n):
            for j in range(n):
                canvas[i*image_size: (i+1)*image_size , j*image_size:(j+1)*image_size] = images[n*i+j].reshape(64,64)
        plt.figure(figsize =(8,8))
        plt.imshow(canvas, cmap ="gray")
        label = "Epoch: {0}".format(order+1)
        plt.xlabel(label)

        if type(order) is str:
            file_name = order
        else:
            file_name = "Mnist_canvas" + str(order)

        plt.savefig(file_name)
        print(os.getcwd())
        print("Image saved in file: ", file_name)
        plt.close()

    def build_net(self):
        self.X = tf.placeholder(tf.float32 , shape = [None, self.length], name ="Input_data")
        self.X_img = tf.reshape(self.X, [-1] + self.data_shape)   # (batch_size , 64, 64, 1)
        self.z = tf.placeholder(tf.float32, shape = [None, self.z_d], name ="latent_var")  #(batch_size , 100)

        self.G = self.Generator(self.z, is_training = True, reuse = False)
        self.D_fake_logits = self.Discriminator(self.G, is_training = True, reuse = False)
        self.D_true_logits = self.Discriminator(self.X_img, is_training = True, reuse = True)

        self.G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self. D_fake_logits, labels = tf.ones_like(self.D_fake_logits)))
        self.D_loss_1 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.D_true_logits , labels = tf.ones_like(self.D_true_logits)))
        self.D_loss_2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.D_fake_logits  , labels = tf.zeros_like(self.D_fake_logits)))
        self.D_loss = self.D_loss_1 + self.D_loss_2

        total_vars = tf.trainable_variables()
        self.d_vars = [var for var in total_vars if  "d_" in var.name]
        self.g_vars = [var for var in total_vars if  "g_" in var.name]


        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.g_optimization = tf.train.AdamOptimizer(learning_rate = self.learning_rate, beta1 = self.beta1).\
                minimize(self.G_loss, var_list = self.g_vars)
            self.d_optimization = tf.train.AdamOptimizer(learning_rate = self.learning_rate, beta1 = self.beta1).\
                minimize(self.D_loss, var_list = self.d_vars)
        print("we successfully make the network")

    def training(self,Data, epoch):  ##(train_data,FLAGS.n_epoch) = ( [55000,4096] , 1 )
        start_time = time.time()
        
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"
        config = tf.ConfigProto(allow_soft_placement = True)
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.5)
        config.gpu_options.allow_growth = True
        
        sess = tf.InteractiveSession(config = config)
        
#         sess = tf.Session()
        sess.run(tf.initialize_all_variables())

        for i in range(epoch):  #epoch
            total_batch = int(len(Data)/self.batch_size)  #55000/64 = 859
            d_value = 0
            g_value = 0
            for j in range(total_batch):  #batch
                batch_xs = Data[j*self.batch_size:j*self.batch_size + self.batch_size]
                z_sampled1 = np.random.uniform(low  = -1.0, high = 1.0, size = [self.batch_size, self.z_d]) #(64, 100)
                Op_d, d_= sess.run([self.d_optimization, self.D_loss], feed_dict = {self.X:batch_xs, self.z: z_sampled1})
                
                z_sampled2 = np.random.uniform(low = -1.0, high = 1.0, size = [self.batch_size, self.z_d])
                Op_g, g_= sess.run([self.g_optimization, self.G_loss], feed_dict = {self.X:batch_xs, self.z: z_sampled2})
                
                self.images_generated = sess.run(self.G, feed_dict = {self.z:z_sampled2})
                
                d_value += d_/total_batch
                g_value +=  g_/ total_batch
            
            self.plot_and_save(i, self.images_generated)
            hour = int((time.time() - start_time)/3600)
            min = int(((time.time() - start_time) - 3600*hour)/60)
            sec = int((time.time()  - start_time) - 3600*hour - 60*min)
            print("Time: ",hour,"h", min,"min",sec ,"sec","   Epoch: ", i, "G_loss: ", g_value, "D_loss: ",d_value)

    def testing(self):
        self.fake_images = self.generator(self.z, is_training = False, reuse = True)
        z_sampled_for_test  = np.random.uniform(low = -1.0, high = 1.0, size = [self.batch_size, self.z_d])
        
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"
        config = tf.ConfigProto(allow_soft_placement = True)
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.5)
        config.gpu_options.allow_growth = True
        
        sess = tf.InteractiveSession(config = config)
        
#         sess = tf.Session()
        sess.run(tf.initialize_all_variables())
        _ = sess.run(self.fake_images, feed_dict  = {self.z: z_sampled_for_test})
        self.plot_and_save("test_image", _)
        print("we have successfully completed the test ")

In [3]:
import tensorflow as tf
# import data
# import model

flags = tf.app.flags

tf.app.flags.DEFINE_string('f', '', 'kernel')

flags.DEFINE_integer("z_d", 100, "Dimension of z")
flags.DEFINE_float("learning_rate", 0.00004, "learning_rate")
flags.DEFINE_integer("batch_size", 64, "batch_size")
flags.DEFINE_integer("n_epoch", 1, "number of epoch")
FLAGS = flags.FLAGS

train_data,train_labels,validation_data,validation_labels,test_data,test_labels = prepare_MNIST_Data()
train_data = preprocessing_data(train_data)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto(allow_soft_placement = True)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.5)
config.gpu_options.allow_growth = True
 
sess = tf.InteractiveSession(config = config)

# sess = tf.Session()
network = Gan(sess,[64,64,1],FLAGS.z_d, FLAGS.learning_rate, FLAGS.batch_size) 
sess.run(tf.initialize_all_variables())
# tf.global_variables_initializer()
network.training(train_data,FLAGS.n_epoch)  #(train_data,FLAGS.n_epoch) = (55000,4096,1)
# network.testing()

Extracting data\train-images-idx3-ubyte.gz
Extracting data\t10k-images-idx3-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
we successfully make the network


C:\Users\user\Anaconda3\envs\pytorch\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


C:\Users\user\python\tensorflow\black\GAN
Image saved in file:  Mnist_canvas0
Time:  0 h 9 min 5 sec    Epoch:  0 G_loss:  10.605354106745432 D_loss:  0.07755599245928453


In [4]:
print('okay')

okay
